In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from scipy.stats import spearmanr
import re
import arviz as az
import pymc as pm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import cellbayesassay as cba
import pickle
import re
import attila_utils

In [2]:
fpath = '../../resources/cell-based-assays/CO28151_outcome-summary_27062023-clean.xlsx'
conc2concentration = pd.read_excel(fpath, 1).rename({'TI nr.': 'TI'}, axis=1)
cN_columns = [c for c in conc2concentration.columns if re.match('^c[1-6]$', c)]
other_columns = [c for c in conc2concentration.columns if not re.match('^c[1-6]$', c)]
conc2concentration = pd.concat([conc2concentration[other_columns], conc2concentration[cN_columns].applymap(lambda x: x * 1e-6)], axis=1)
conc2concentration['VC'] = conc2concentration.apply(lambda r: r.loc['c6'] * 1e-3 if not np.isnan(r.loc['c6']) else r.loc['c3'] * 1e-3, axis=1)
conc2concentration['LPS'] = conc2concentration['VC']
conc2concentration = conc2concentration.set_index('TI')
conc2concentration

,TI ID,name,c1,c2,c3,c4,c5,c6,VC,LPS
TI,,,,,,,,,,
TI1,2542,Amiloride,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI2,2543,Phenamil methanesulfonate salt,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI3,2538,Benzamil hydrochloride hydrate,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI4,2544,"5-(N,N-Dimethyl)amiloride hydrochloride",0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI5,2540,5-(N-Ethyl-N-isopropyl)amiloride,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI6,2541,"5-(N,N-Hexamethylene)amiloride",0.000010,0.000001,2.000000e-07,NaN,NaN,NaN,2.000000e-10,2.000000e-10
TI7,2539,5-(N-Methyl-N-isobutyl)amiloride,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI8,2719,NS 383,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09
TI9,2703,A-317567,0.000100,0.000010,1.000000e-06,NaN,NaN,NaN,1.000000e-09,1.000000e-09


In [3]:
experimentl = [
    'Aβ clearance (BV2 cells)',
    'Aβ release (H4 cells)',
    'Trophic factor withdrawal (primary neurons)',
    'LPS neuroinflammation (BV2 cells)',
    'Tau phoshorylation'
              ]
experimentd = dict(zip(experimentl, np.arange(len(experimentl))))
treatmentd = {
    'TI11': 'DIM-C-pPhOCH3',
    'TI12': 'C-DIM12',
}
fpath = '../../resources/cell-based-assays/CO28151_data-summary-clean.xlsx'

In [4]:
def extract_TI_exp_data(experiment, TI, control_TI='VC', batchvars=['Batch', 'Plate'], experimentd=experimentd, fpath=fpath):
    data = pd.read_excel(fpath, experimentd[experiment])
    TI_data = data.loc[data.TI == TI]
    # ensure that all treatment TI data are from the same batch:plate
    if len(TI_data.groupby(batchvars)) != 1:
        print('treatment with multiple batches')
        return(None)
    b = data.Plate == TI_data.iloc[0].loc['Plate']
    # if there's no information on the
    if not TI_data.iloc[0].isna().loc['Batch']:
        b = b & (data.Batch == data.iloc[0].loc['Batch'])
    TI_exp_data = data.loc[b]
    TI_exp_data_control = TI_exp_data.loc[TI_exp_data.TI == control_TI].copy()
    # if there's no control for the same batch:plate, use controls from all other batch:plate combinations
    if len(TI_exp_data_control) == 0:
        TI_exp_data_control = data.loc[data.TI == control_TI].copy()
    TI_exp_data_control['conc'] = control_TI
    TI_exp_data_TI = TI_exp_data.loc[TI_exp_data.TI == TI].copy()
    TI_exp_data = pd.concat([TI_exp_data_control, TI_exp_data_TI], axis=0)
    return(TI_exp_data)

experiment = experimentl[0]
TI = list(treatmentd.keys())[0]


In [5]:
datad = dict()

experiment = experimentl[0] #'Aβ clearance (BV2 cells)'
TI_list = ['TI11', 'TI12']
datad[experiment] = {TI: extract_TI_exp_data(experiment, TI, control_TI='VC') for TI in TI_list}

experiment = experimentl[1] #'Aβ release (H4 cells)'
TI_list = ['TI11', 'TI12']
datad[experiment] = {TI: extract_TI_exp_data(experiment, TI, control_TI='VC') for TI in TI_list}

experiment = experimentl[2] #'Trophic factor withdrawal (primary neurons)'
TI_list = ['TI11', 'TI12']
datad[experiment] = {TI: extract_TI_exp_data(experiment, TI, control_TI='VC') for TI in TI_list}

experiment = experimentl[3] #'LPS neuroinflammation (BV2 cells)'
TI_list = ['TI11', 'TI12']
datad[experiment] = {TI: extract_TI_exp_data(experiment, TI, control_TI='LPS') for TI in TI_list}

experiment = experimentl[4] #'Tau phoshorylation'
TI_list = ['TI11', 'TI12']
datad[experiment] = {TI: extract_TI_exp_data(experiment, TI, control_TI='VC') for TI in TI_list}

datad[experiment][TI_list[0]]

,Batch,Plate,IRN,TI,conc,Tau [pg/ml],pTau (T231) [AU],BCA total protein,Tau [pg/µg],pTau (T231) [AU2],Ratio pTau /Tau
348,1,3,VC_3_1,VC,VC,408.4,75.2,256.8,1.6,47.3,29.7
349,1,3,VC_3_2,VC,VC,331.0,74.1,208.8,1.6,46.7,29.5
350,1,3,VC_3_3,VC,VC,386.4,70.8,243.2,1.6,44.6,28.1
351,1,3,VC_3_4,VC,VC,412.1,76.4,251.5,1.6,46.6,28.5
352,1,3,VC_3_5,VC,VC,457.5,87.2,281.2,1.6,53.6,33.0
353,1,3,VC_3_6,VC,VC,410.8,72.2,268.1,1.5,47.1,30.8
180,1,3,TI11 c1_1,TI11,c1,422.7,83.0,289.5,1.5,56.8,38.9
181,1,3,TI11 c1_2,TI11,c1,426.0,79.4,231.5,1.8,43.1,23.4
182,1,3,TI11 c1_3,TI11,c1,356.3,67.6,228.8,1.6,43.4,27.9
183,1,3,TI11 c1_4,TI11,c1,439.1,88.9,261.6,1.7,53.0,31.5


In [6]:
datad[experiment][TI_list[0]].conc.apply(lambda x: conc2concentration.loc[TI, x])

348    2.000000e-10
349    2.000000e-10
350    2.000000e-10
351    2.000000e-10
352    2.000000e-10
353    2.000000e-10
180    1.000000e-05
181    1.000000e-05
182    1.000000e-05
183    1.000000e-05
184    1.000000e-05
185    1.000000e-05
186    1.000000e-06
187    1.000000e-06
188    1.000000e-06
189    1.000000e-06
190    1.000000e-06
191    1.000000e-06
192    2.000000e-07
193    2.000000e-07
194    2.000000e-07
195    2.000000e-07
196    2.000000e-07
197    2.000000e-07
Name: conc, dtype: float64

In [7]:
%connect_info

{
  "shell_port": 50552,
  "iopub_port": 50553,
  "stdin_port": 50554,
  "control_port": 50556,
  "hb_port": 50555,
  "ip": "127.0.0.1",
  "key": "61dbacfc-98abdd43e8ddeba32e6c61d2",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "python3"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-ce7a3bbf-236f-49aa-ae3c-9a6dfccbb5ac.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
